*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
print('Uninstalling tf-nightly and related packages...')
!pip uninstall -y tf-nightly protobuf h5py keras-nightly tb-nightly tensorflow
print('Uninstallation complete.')

Uninstalling tf-nightly and related packages...
Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
Found existing installation: h5py 3.15.1
Uninstalling h5py-3.15.1:
  Successfully uninstalled h5py-3.15.1
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/req/req_install.py", line 715, in uninstall
    dist = get_default_environment().get_distribution(self.req.name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/metadata/importlib/_envs.py", l

In [ ]:
print('Installing tensorflow==2.19.0...')
!pip install tensorflow==2.19.0
print('Installation complete.')

Installing tensorflow==2.19.0...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 23.2 MB/s eta 0:00:00


Installation complete.


In [ ]:
# Re-import libraries to check the installed version
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.19.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-11-10 11:50:17--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2025-11-10 11:50:18 (25.3 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-11-10 11:50:18--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
df_train = pd.read_csv(train_file_path, sep = '\t', header = None)
df_test = pd.read_csv(test_file_path, sep = '\t', header = None)
df_train.head(3)

,0,1
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# 0 = ham, 1 = spam
df_train[0] = le.fit_transform(df_train[0])
df_train.head()

,0,1
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
df_test[0] = le.fit_transform(df_test[0])
df_test.head()

,0,1
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?


In [ ]:
df_test.dtypes

,0
0,int64
1,object


In [ ]:
train_y = df_train[0].to_numpy()
slice_x  = df_train[1].to_numpy()
train_ds = tf.data.Dataset.from_tensor_slices((slice_x, train_y))

In [ ]:
test_y = df_test[0].to_numpy()
test_slice_x = df_test[1].to_numpy()
test_ds = tf.data.Dataset.from_tensor_slices((test_slice_x, test_y))

In [ ]:
#batching the tensorslices

BUFFER_SIZE = 100
BATCH_SIZE = 64

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train_ds.element_spec

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [ ]:
#text encoder
encoder = tf.keras.layers.TextVectorization(
    max_tokens = 1000,
    output_mode = 'int',
    output_sequence_length = 1000
)
encoder.adapt(train_ds.map(lambda text, label : text ))

In [ ]:
word_list = np.array(encoder.get_vocabulary())
print(word_list)

['' '[UNK]' 'to' 'i' 'you' 'a' 'the' 'u' 'and' 'in' 'is' 'me' 'my' 'for'
 'your' 'of' 'it' 'call' 'have' 'on' 'that' 'are' 'now' 'im' '2' 'not'
 'but' 'so' 'at' 'or' 'do' 'can' 'with' 'be' 'will' 'if' 'get' 'ur' 'just'
 'we' 'this' 'no' 'its' 'up' 'go' '4' 'ok' 'dont' 'ltgt' 'free' 'when'
 'out' 'how' 'all' 'what' 'from' 'know' 'like' 'got' 'then' 'come' 'good'
 'ill' 'time' 'am' 'was' 'only' 'day' 'he' 'love' 'send' 'there' 'as'
 'want' 'text' 'going' 'by' 'ü' 'one' 'about' 'need' 'txt' 'lor' 'still'
 'our' 'see' 'n' 'r' 'home' 'stop' 'sorry' 'reply' 'back' 'today' 'tell'
 'she' 'new' 'been' 'mobile' 'later' 'take' 'hi' 'da' 'her' 'they' 'well'
 'think' 'night' 'has' 'any' 'please' 'much' 'cant' 'week' 'dear' 'an'
 'happy' 'some' 'pls' 'hey' 'did' 'phone' 'oh' 'who' 'too' 'here' 'make'
 'great' 'where' 'way' 'him' 'work' 'tomorrow' 'd' 'more' 'hope' 'had'
 'say' 'claim' 'wat' 'thats' 'number' 'give' 'should' 'really' 'ask'
 'already' 'yes' 'were' 'amp' 'doing' 'after' 'yeah' 'right' '

In [ ]:
model = keras.Sequential([
                       encoder,
                       tf.keras.layers.Embedding(input_dim = len(encoder.get_vocabulary()), output_dim = 64, mask_zero = True),
                       tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                       tf.keras.layers.Dense(64, activation = 'relu'),
                       tf.keras.layers.Dense(1)
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
model.fit(train_ds, validation_data = test_ds, epochs = 5, validation_steps = 30)

Epoch 1/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.8886 - loss: 0.3899 - val_accuracy: 0.9749 - val_loss: 0.0964
Epoch 2/5


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


66/66 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.9826 - loss: 0.0785 - val_accuracy: 0.9842 - val_loss: 0.0622
Epoch 3/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.9912 - loss: 0.0489 - val_accuracy: 0.9849 - val_loss: 0.0792
Epoch 4/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.9949 - loss: 0.0421 - val_accuracy: 0.9842 - val_loss: 0.0900
Epoch 5/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9968 - loss: 0.0383 - val_accuracy: 0.9828 - val_loss: 0.1301


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # pred_text needs to be explicitly converted to a TensorFlow string tensor
  result = model.predict(tf.constant([pred_text]))
  probability = result[0][0]
  if probability >= 0.5:
    p_class = 'spam'
  else:
    p_class = 'ham'

  prediction = [probability, p_class]




  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
[np.float32(-0.1264622), 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
You passed the challenge. Great job!
